<a href="https://colab.research.google.com/github/srkiyengar/PycharmProjects/blob/master/oreo_pybullet_habitat_deepgaze2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%bash

DIRECTORY=/content/oreo-pybullet
if [ -d "$DIRECTORY" ]; then
  rm -r /content/oreo-pybullet
fi
mkdir /content/oreo-pybullet
cp /content/drive/My\ Drive/Colab_Notebooks/Oreo-habitat-integration/* /content/oreo-pybullet/  -r

In [ ]:
%tensorflow_version 1.13.1

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.13.1`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [ ]:
!curl -L https://raw.githubusercontent.com/facebookresearch/habitat-sim/master/examples/colab_utils/colab_install.sh | NIGHTLY=false bash -s
!wget -c http://dl.fbaipublicfiles.com/habitat/mp3d_example.zip && unzip -o mp3d_example.zip -d /content/habitat-sim/data/scene_datasets/mp3d/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2658  100  2658    0     0  14932      0 --:--:-- --:--:-- --:--:-- 14932
--2021-11-04 15:57:30--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2021-11-04 15:57:30--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response..

In [ ]:
pip install 'pybullet==2.8.7'

     |████████████████████████████████| 100.7 MB 15 kB/s             


In [ ]:
import sys
_ = (sys.path.append("/content/oreo-pybullet"))

In [ ]:
import os
if "google.colab" in sys.modules:
    # This tells imageio to use the system FFMPEG that has hardware acceleration.
    os.environ["IMAGEIO_FFMPEG_EXE"] = "/usr/bin/ffmpeg"

In [ ]:
import tensorflow as tf
print(tf.__version__)

1.15.2


In [ ]:
import agent_oreo as ao
import saliency as s
import cv2
pybullet_sim = ao.OreoPyBulletSim("/content/oreo-pybullet/")

INFO:LOG_LEVEL: DEBUG
INFO:There are 16 links in file

DEBUG:base_joint 0
DEBUG:com frame (posn (global) & orn (global)) (0.0, 0.0, 0.0) (0.26947353370204224, -0.1483647897014979, -0.336452277174072)
DEBUG:inertial offset (posn (link frame), orn (link frame)) (0.0, 0.0, 0.0) (0.2694735337020423, -0.1483647897014979, -0.33645227717407206)
DEBUG:link frame (posn (global), orn (global)) (0.0, 0.0, 0.0) (0.0, -0.0, 0.0)
DEBUG:type 4
DEBUG:axis (0.0, 0.0, 0.0)
DEBUG:collision (posn (COM frame), orn(COM frame)) (0.0, 0.0, 0.0) (-0.2110668264094377, 0.22426388413035764, 0.29244941472167946)
DEBUG:visual (posn (link frame), orn(link frame)) (0.0, 0.0, 0.0) (0.0, -0.0, 0.0)


DEBUG:neck_joint 1
DEBUG:com frame (posn (global) & orn (global)) (0.03609000000000001, -0.00028000000000000507, 0.07360000000000004) (0.013334272492713626, -0.009866778134608222, 0.02516708744634716)
DEBUG:inertial offset (posn (link frame), orn (link frame)) (0.00028, 0.00039, 0.07587) (0.013334272492713562, -0.009866778

oreo_sim constructed


DEBUG:inertial offset (posn (link frame), orn (link frame)) (0.03025, 0.0, 0.0) (0.0, -0.0, 0.0)
DEBUG:link frame (posn (global), orn (global)) (0.07711464911699295, 0.013830000534653664, 0.22889283299446106) (-0.00018403216069739608, 0.2591799959642588, -5.140327524362407e-05)
DEBUG:type 2
DEBUG:axis (0.0, 0.0, 0.0)
DEBUG:collision (posn (COM frame), orn(COM frame)) (0.0, 0.0, 0.0) (0.0, -0.0, 0.0)
DEBUG:visual (posn (link frame), orn(link frame)) (0.03025, 0.0, 0.0) (0.0, -0.0, 0.0)


DEBUG:dogbone_joint_mid_right 10
DEBUG:com frame (posn (global) & orn (global)) (0.10635431232199295, -0.015170000000000008, 0.221140114893213) (4.7715550532455413e-17, 0.25917999999999974, -3.961440248693532e-17)
DEBUG:inertial offset (posn (link frame), orn (link frame)) (0.03025, 0.0, 0.0) (0.0, -0.0, 0.0)
DEBUG:link frame (posn (global), orn (global)) (0.07711464911699295, -0.015169999562203884, 0.22889283299446106) (4.771555144378362e-17, 0.25917999843751466, -3.9614403165568695e-17)
DEBUG:type 2
D

In [ ]:
from google.colab.patches import cv2_imshow
import quaternion
import numpy as np

#scene ="/content/habitat-sim/data/scene_datasets/habitat-test-scenes/van-gogh-room.glb"
scene = "/content/habitat-sim/data/scene_datasets/habitat-test-scenes/skokloster-castle.glb"
result_folder = "/content/oreo-pybullet/saliency"
oreo_in_habitat = ao.agent_oreo(scene, result_folder, depth_camera=False, loc_depth_cam = 'c', foveation=False)

I1104 16:06:25.012912 77 simulator.py:221] Loaded navmesh /content/habitat-sim/data/scene_datasets/habitat-test-scenes/skokloster-castle.navmesh
INFO:I1104 16:06:25.012912 77 simulator.py:221] Loaded navmesh /content/habitat-sim/data/scene_datasets/habitat-test-scenes/skokloster-castle.navmesh


Agent rotation quaternion(1, 0, 2.45858027483337e-05, 0) Agent position [-1.7926959   0.11083889 19.255245  ]


In [ ]:
from google.colab import widgets
import ipywidgets
from IPython.display import display, clear_output, Image


action_name = ['backward', 'forward', 'left', 'right', 'up', 'down', 'cw', 'ccw','Default', 'quit', 'New Image', 'Last Position' ]
total_actions = len(action_name)
print (total_actions)


# register function to be called when the button is clicked

def on_button_clicked(selection):
  # Display the message within the output widget.
  with output:
    clear_output()
    take_action(selection)
    

12


In [ ]:
# Take action based on input

delta_move = 0.1
delta_ang_ccw  = quaternion.from_rotation_vector([0.0, 2*np.pi/30,0.0])
delta_ang_cw = quaternion.from_rotation_vector([0.0, -2*np.pi/30, 0.0])
ang_quat = quaternion.from_rotation_vector([0.0, 0.0, 0.0])

def take_action(selected_key):
  print(f'Last Action {selected_key.description}')
  if selected_key.description == 'backward':
    oreo_in_habitat.move_and_rotate_agent(ang_quat, [0.0, 0.0, delta_move])
  elif selected_key.description == 'forward':
    oreo_in_habitat.move_and_rotate_agent(ang_quat, [0.0, 0.0, -delta_move])
  elif selected_key.description == 'left':
    oreo_in_habitat.move_and_rotate_agent(ang_quat, [-delta_move, 0.0, 0.0])
  elif selected_key.description == 'right':
    oreo_in_habitat.move_and_rotate_agent(ang_quat, [delta_move, 0.0, 0.0])
  elif selected_key.description == 'up':
    oreo_in_habitat.move_and_rotate_agent(ang_quat, [0.0, delta_move, 0.0])
  elif selected_key.description == 'ccw':
    oreo_in_habitat.move_and_rotate_agent(delta_ang_ccw, [0.0, 0.0, 0.0])  
  elif selected_key.description == 'cw':
    oreo_in_habitat.move_and_rotate_agent(delta_ang_cw, [0.0, 0.0, 0.0])
  elif selected_key.description == 'Default':
    oreo_in_habitat.reset_state()
  elif selected_key.description == 'New Image':
    #saved images with unique name based scene and on timestamp, ending with RGB0
    oreo_in_habitat.capture_start_image_for_saliency_colab()  
  elif selected_key.description  == 'quit':
    #oreo_in_habitat.sim.close()
    pass
  else:
    pass
  # display the left rgb camera image  
  cv2_imshow(oreo_in_habitat.my_images[0][..., 0:3][..., ::-1])

# create output widget
output = ipywidgets.widgets.Output()

# create a list of buttons, one for each star sign
button_list = []
for i in range(0, total_actions):
  button = ipywidgets.widgets.Button(description = action_name[i])
  button.on_click(on_button_clicked)
  button_list.append(button) 

print('\nMove Oreo\'s view by clicking one of the buttons\n')

# arrange the sign images and buttons into two rows
grid = widgets.Grid(1, 6, header_row=True, header_column=True) 
for (row, col) in grid:
  index = row*6+col
  display(button_list[index])
for (row, col) in grid:
  index = row*6+col
  display(button_list[index+6])

# display output

display(output)


Move Oreo's view by clicking one of the buttons



,,,,,


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Button(description='Last Position', style=ButtonStyle())

<IPython.core.display.Javascript object>

Output()

Process for saliency. This object will be initially devoid of saliency map and this has to added to the object after getting the saliency map from Deep Gaze II.

In [ ]:
from scipy.ndimage import zoom
from scipy.special import logsumexp   #changed from scipy.misc to scipy.special
import matplotlib.pyplot as plt
import saliency
import pickle


In [ ]:
# load precomputed log density over a 1024x1024 image
centerbias_template = np.load('/content/oreo-pybullet/centerbias.npy')
# Producing a processed file containing saliency heatmap from an image file
def produce_and_process_saliency_file(filename):
  my_sal_process = saliency.process_image(filename, output_subfolder="no")
  img = my_sal_process.imageL
  lefteye_orn_wrt_WCS = my_sal_process.lefteye
  focal_distance = my_sal_process.focal_distance
  # rescale to match image size
  centerbias = zoom(centerbias_template, (img.shape[0]/1024, img.shape[1]/1024), order=0, mode='nearest')
  # renormalize log density
  centerbias -= logsumexp(centerbias)
  image_data = img[np.newaxis, :, :, :]  # BHWC, three channels (RGB)
  centerbias_data = centerbias[np.newaxis, :, :, np.newaxis]  # BHWC, 1 channel (log density)
  tf.get_default_graph()
  tf.reset_default_graph()
  check_point = '/content/oreo-pybullet/DeepGazeII.ckpt'  # DeepGaze II
  #check_point = 'ICF.ckpt'  # ICF
  new_saver = tf.train.import_meta_graph('{}.meta'.format(check_point))
  input_tensor = tf.get_collection('input_tensor')[0]
  centerbias_tensor = tf.get_collection('centerbias_tensor')[0]
  log_density = tf.get_collection('log_density')[0]
  log_density_wo_centerbias = tf.get_collection('log_density_wo_centerbias')[0]
  with tf.Session() as sess:
    new_saver.restore(sess, check_point)
    log_density_prediction = sess.run(log_density, {input_tensor: image_data,centerbias_tensor: centerbias_data,})
  salmapL = np.exp(log_density_prediction[0,:,:,0])
  my_sal_process.load_salmap(salmapL)
  #my_sal_process.save_salmap()
  my_sal_process.process_saliency()
  f = my_sal_process.save_all()
  return f, img, salmapL, lefteye_orn_wrt_WCS, focal_distance


def write_file(myfilename, info):
    try:
        with open(myfilename, "wb") as f:
            pickle.dump(info, f)
            print(f"Saved saliency file {myfilename}")
            return myfilename
    except BaseException as err:
      print(f"Error: {err}, {type(err)}")
      print(f"Failure in saving saliency file {myfilename}")
      return None


def save_salmap_list(s_picklefile, salmap_list):
  
  heatmap_list = []
  for heatmap in salmap_list:
    a = heatmap.flatten()
    heatmap_list.append(a)
  write_file(s_picklefile,heatmap_list)


def generate_ten_salmaps(file, s_partname):
  count = 0
  count_max = 11
  image_list = []
  salmap_list = []
  IOR_list = []
  head, tail = os.path.split(file)
  d = len(s_partname)
  sal_pickle_file = head + "/" + s_partname[0:d-1] + "-salmap-list"
  imagefile = file
  while count < count_max:
    pfile, imageL, salmapL, lefteye_orn, fd = produce_and_process_saliency_file(imagefile)
    imagefile = oreo_in_habitat.capture_next_image_from_fixations_colab(pfile, pybullet_sim, IOR_list, fd, count)
    image_list.append(imageL)
    salmap_list.append(salmapL)
    IOR_list.append(lefteye_orn)  
    if imagefile is None:
      count = count_max
    count +=1
  save_salmap_list(sal_pickle_file, salmap_list)
  return

In [ ]:
import os, shutil


for path, subdirs, files in os.walk(result_folder):
    for filename in files:
      d = filename.find("^")
      dir_name = filename[0:3] + filename[d+1:]
      new_dir = path + '/' + dir_name
      if not os.path.exists(new_dir):
        os.makedirs(new_dir)
      file = path + '/' + filename
      shutil.move(file, new_dir)
      newfile = new_dir + '/' +filename
      generate_ten_salmaps(newfile, dir_name)
    break # we are not interested in sub-directories of the result_folder

      

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.


'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.


INFO:tensorflow:Restoring parameters from /content/oreo-pybullet/DeepGazeII.ckpt


INFO:Restoring parameters from /content/oreo-pybullet/DeepGazeII.ckpt


Saved processed saliency file /content/oreo-pybullet/saliency/sko2021-11-04-16-09-11RGB0/skokloster-castle.glb^2021-11-04-16-09-11RGB0-sal-processed
Total number of previous frames 0
Proposed fixation 1: Pixel location (232,248) is a new fixation from the first (1st) saccade
Agent Rotation quaternion(-0.978152692317963, -0, 0.207887649536133, 4.39237008081496e-16)
Desire to move to yawL = 5.355825042855193, pitchL = 88.21789868689504
Desire to move to yawR = 5.355825042855193, pitchR = 88.21789868689504
Moving Sensors to new position
After rotation Left Eye yaw = 5.359906082890401 pitch = 88.21894141908521
After rotation Right Eye yaw = 5.359251932592072 pitch = 88.2186417755626
Saved Image file /content/oreo-pybullet/saliency/sko2021-11-04-16-09-11RGB0/skokloster-castle.glb^2021-11-04-16-09-11RGB1
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.


...

Saved processed saliency file /content/oreo-pybullet/saliency/sko2021-11-04-16-09-11RGB0/skokloster-castle.glb^2021-11-04-16-09-11RGB9-sal-processed
Total number of previous frames 9
Fixation Center of frame 0 in current frame = (280, 327)
Proposed fixation 10 was NOT fixated in saccade 1
Fixation Center of frame 1 in current frame = (255, 319)
!!! Proposed fixation 10 IOR: Pixel location (248,312) was fixated in saccade 2
Total number of previous frames 9
Fixation Center of frame 0 in current frame = (280, 327)
Proposed fixation 10 was NOT fixated in saccade 1
Fixation Center of frame 1 in current frame = (255, 319)
Proposed fixation 10 was NOT fixated in saccade 2
Fixation Center of frame 2 in current frame = (247, 327)
Proposed fixation 10 was NOT fixated in saccade 3
Fixation Center of frame 3 in current frame = (238, 318)
Proposed fixation 10 was NOT fixated in saccade 4
Fixation Center of frame 4 in current frame = (230, 327)
Proposed fixation 10 was NOT fixated in saccade 5
Fixa

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.


INFO:tensorflow:Restoring parameters from /content/oreo-pybullet/DeepGazeII.ckpt


INFO:Restoring parameters from /content/oreo-pybullet/DeepGazeII.ckpt


Saved processed saliency file /content/oreo-pybullet/saliency/sko2021-11-04-16-09-11RGB0/skokloster-castle.glb^2021-11-04-16-09-11RGB10-sal-processed
Total number of previous frames 10
Fixation Center of frame 0 in current frame = (272, 391)
Proposed fixation 11 was NOT fixated in saccade 1
Fixation Center of frame 1 in current frame = (245, 381)
Proposed fixation 11 was NOT fixated in saccade 2
Fixation Center of frame 2 in current frame = (236, 391)
Proposed fixation 11 was NOT fixated in saccade 3
Fixation Center of frame 3 in current frame = (227, 380)
Proposed fixation 11 was NOT fixated in saccade 4
Fixation Center of frame 4 in current frame = (217, 390)
Proposed fixation 11 was NOT fixated in saccade 5
Fixation Center of frame 5 in current frame = (207, 399)
Proposed fixation 11 was NOT fixated in saccade 6
Fixation Center of frame 6 in current frame = (222, 318)
Proposed fixation 11 was NOT fixated in saccade 7
Fixation Center of frame 7 in current frame = (231, 311)
Proposed 

In [ ]:
cp /content/oreo-pybullet/saliency  /content/drive/My\ Drive/Colab_Notebooks/Oreo-habitat-integration/ -r

In [ ]:
oreo_in_habitat.sim.close()